<a href="https://colab.research.google.com/github/limshaocong/SysBERT/blob/main/MLM_TAPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

Install HuggingFace transformers and datasets libaries.

In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.1 MB/s 
     |████████████████████████████████| 4.2 MB 47.8 MB/s 
     |████████████████████████████████| 1.1 MB 50.1 MB/s 
     |████████████████████████████████| 140 kB 76.4 MB/s 
     |████████████████████████████████| 86 kB 3.9 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 212 kB 63.6 MB/s 
     |████████████████████████████████| 596 kB 63.6 MB/s 
     |████████████████████████████████| 127 kB 78.0 MB/s 
     |████████████████████████████████| 6.6 MB 55.1 MB/s 
     |████████████████████████████████| 94 kB 903 kB/s 
     |████████████████████████████████| 271 kB 65.8 MB/s 
     |████████████████████████████████| 144 kB 63.4 MB/s 
     |████████████████████████████████| 112 kB 67.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling

In [2]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-062fd662-19a9-e589-77e0-f5a44b212a3c)


# Import and Pre-process Data


In [3]:
from datasets import load_dataset

ds = load_dataset('limsc/mlm-tapt-requirements')

ds

Downloading:   0%|          | 0.00/805 [00:00<?, ?B/s]

Using custom data configuration limsc--reqbert-mlm-tapt-a68ae7ead0db5cc7


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/limsc___parquet/limsc--reqbert-mlm-tapt-a68ae7ead0db5cc7/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'reqs'],
        num_rows: 37797
    })
    val: Dataset({
        features: ['source', 'reqs'],
        num_rows: 1990
    })
})

In [4]:
from transformers import AutoTokenizer

# model_checkpoint = 'bert-base-cased'
model_checkpoint = 'roberta-base'
# model_checkpoint = 'allenai/scibert_scivocab_cased'

tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    use_fast = True
)

def encode(requirements):
    return tokenizer(requirements['reqs'], truncation = False)

ds_tokenized = ds.map(
    encode,
    batched = True,
    remove_columns = ['source', 'reqs']
)

ds_tokenized

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (733 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 37797
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1990
    })
})

In [5]:
chunk_size = 128

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result['labels'] = result['input_ids'].copy()
    
    return result

In [6]:
ds_chunked = ds_tokenized.map(group_texts, batched = True)
ds_chunked

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 12204
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 650
    })
})

In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm_probability = 0.15
)

In [10]:
batch_size = 16

batched_train_ds = ds_chunked['train'].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'labels'],
    collate_fn = data_collator,
    shuffle = True,
    batch_size = batch_size,
)

batched_val_ds = ds_chunked['val'].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'labels'],
    collate_fn = data_collator,
    shuffle = False,
    batch_size = batch_size,
)

# Task-Adpative Pre-Training (TAPT)

In [11]:
import tensorflow as tf
from transformers import TFAutoModelForMaskedLM, create_optimizer

tf.keras.utils.set_random_seed(1)
num_epochs = 30

def create_model():
  
  model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

  num_train_steps = len(batched_train_ds) * num_epochs

  optimizer, schedule = create_optimizer(
      init_lr = 2e-5,
      num_warmup_steps = 1000,
      num_train_steps = num_train_steps,
      weight_decay_rate = 0.01,
  )

  model.compile(optimizer = optimizer)

  return model

model = create_model()

model.summary()

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Model: "tf_roberta_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 lm_head (TFRobertaLMHead)   multiple                  39642969  
                                                                 
Total params: 124,697,433
Trainable params: 124,697,433
Non-trainable params: 0
_________________________________________________________________


In [12]:
import math
import os
from tensorflow.keras.callbacks import Callback, CSVLogger, ModelCheckpoint
from transformers import PushToHubCallback

class Perplexity(Callback):

    def __init__(self, validation):
        super(Perplexity, self).__init__()
        self.validation = validation

    def on_epoch_end(self, epoch, logs = {}):

        val_score = self.model.evaluate(self.validation)     
        val_perplexity = math.exp(val_score)
        logs['val_perplexity'] = val_perplexity

perplexity_cb = Perplexity(batched_val_ds)

csvlogger_cb = CSVLogger('roberta-results.csv')

checkpoint_path = 'roberta-checkpoints/reqbert-mlm-epoch{epoch}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

modelcheckpoint_cb = ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only = True,
    verbose = 1
)

In [13]:
callbacks = [perplexity_cb, csvlogger_cb, modelcheckpoint_cb]

In [14]:
model.fit(
    batched_train_ds,
    validation_data = batched_val_ds,
    epochs = num_epochs,
    callbacks = callbacks
)

Epoch 1/30
41/41 [==============================] - 5s 125ms/step - loss: 2.0316

Epoch 1: saving model to model_checkpoints/reqbert-mlm-epoch1.ckpt
762/762 [==============================] - 303s 367ms/step - loss: 2.4130 - val_loss: 2.0196 - val_perplexity: 7.6266
Epoch 2/30
41/41 [==============================] - 5s 125ms/step - loss: 1.8820

Epoch 2: saving model to model_checkpoints/reqbert-mlm-epoch2.ckpt
762/762 [==============================] - 275s 361ms/step - loss: 2.0484 - val_loss: 1.8536 - val_perplexity: 6.5666
Epoch 3/30
41/41 [==============================] - 5s 125ms/step - loss: 1.7634

Epoch 3: saving model to model_checkpoints/reqbert-mlm-epoch3.ckpt
762/762 [==============================] - 275s 361ms/step - loss: 1.8940 - val_loss: 1.7387 - val_perplexity: 5.8324
Epoch 4/30
41/41 [==============================] - 5s 127ms/step - loss: 1.7016

Epoch 4: saving model to model_checkpoints/reqbert-mlm-epoch4.ckpt
762/762 [==============================] - 274s 36

KeyboardInterrupt: ignored

In [16]:
# ! huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/huggingface_hub/commands/user.py", line 176, in run
    token = getpass("Token: 

In [ ]:
# os.listdir(checkpoint_dir)
# latest_cp = tf.train.latest_checkpoint(checkpoint_dir)
# # chosen_cp = 

# # Reinstate checkpoint weights
# model = create_model()
# model.load_weights(latest_cp)

# # Re-evaluate model on validation data
# model.evaluate(batched_val_ds)